In [5]:
from bs4 import BeautifulSoup
import requests
import json
import time
import datetime as dt
import re

In [19]:
# Dealers description
dealer_dct = {
    'id': 'khushiauto',
    'name': 'Khushi Auto',
    'street': '2783 Derry Rd E.',
    'city': 'Mississauga',
    'zip': 'L4T1A3',
    'province': 'ON',
    'phone': '905-460-9624',
    'url': 'https://www.khushiauto.com',
    'latitude': '43.700880',
    'longitude': '-79.646490'
}

print(json.dumps(dealer_dct, indent=4))

with open('data2/khushiauto.json', 'w', encoding='utf-8') as fp:
    json.dump(dealer_dct, fp, indent=4)

{
    "id": "khushiauto",
    "name": "Khushi Auto",
    "street": "2783 Derry Rd E.",
    "city": "Mississauga",
    "zip": "L4T1A3",
    "province": "ON",
    "phone": "905-460-9624",
    "url": "https://www.khushiauto.com",
    "latitude": "43.700880",
    "longitude": "-79.646490"
}


In [6]:
base_url ='https://www.khushiauto.ca/'
inventory_url = 'https://www.khushiauto.ca/used-cars'

In [7]:
def parse_inventory(query_args):
    global base_url, inventory_url
    res = []

    time.sleep(1.1)
    response = requests.get(inventory_url, query_args)
    
    soup = BeautifulSoup(response.text, 'lxml')
    vehicles = soup.find_all('div', class_='vehicle search-result-item vehicleList')
    if vehicles is None:
        return res, False
    
    for v in vehicles:
        # check if it isn't sold
        sold = v.find('div', class_='SoldVehicle')
        if sold is not None:
            return res, False # all the rest of the inventory is a list of sold vehicles, no need to go further

        # get vehicle description page url
        url = base_url + v.div.contents[1].div.div.a['href']
        
        # Printing URL (for debugginh purposes)
        print(url)
        res.append(url)
    
    return res, True

In [8]:
# Making a list of inventory URLs
vehicle_urls = []
cpage = 1

urls, has_more = parse_inventory({'ppage': 100, 'cpage': cpage})

while has_more:
    vehicle_urls.extend(urls)
    cpage += 1
    urls, has_more = parse_inventory({'cpage': cpage})
    
vehicle_urls.extend(urls)
print(len(vehicle_urls))

https://www.khushiauto.ca//cars/used/2016-Nissan-Altima-Sedan-114128
https://www.khushiauto.ca//cars/used/2017-Hyundai-Elantra-Sedan-114118
https://www.khushiauto.ca//cars/used/2012-Mercedes-Benz-CL-Class-Coupe-113364
https://www.khushiauto.ca//cars/used/2015-Acura-ILX-Sedan-113362
https://www.khushiauto.ca//cars/used/2009-Honda-Accord-Sedan-Sedan-111398
https://www.khushiauto.ca//cars/used/2010-Nissan-Altima-Coupe-111392
https://www.khushiauto.ca//cars/used/2008-Nissan-Rogue-SUV-75061
https://www.khushiauto.ca//cars/used/2014-Audi-Q5-SUV-97354
https://www.khushiauto.ca//cars/used/2014-BMW-3-Series-Sedan-75072
https://www.khushiauto.ca//cars/used/2012-Mercedes-Benz-C-Class-Coupe-101233
https://www.khushiauto.ca//cars/used/2014-BMW-3-Series-Sedan-108795
https://www.khushiauto.ca//cars/used/2014-Chevrolet-Cruze-Sedan-103052
https://www.khushiauto.ca//cars/used/2014-Nissan-Pathfinder-SUV-104208
https://www.khushiauto.ca//cars/used/2011-Land-Rover-Range-Rover-SUV-100807
https://www.khushia

In [17]:
# Function parses vehicle description page and returns a dictionary with its parameters
def parse_car_infopage(url):
    # Translation table from their names to our key names
    translation_table = {
        'Make:': 'make',
        'Model:': 'model',
        'Year:': 'year',
        'Body Style:': 'body_style',
        'Odometer:': 'mileage', # Note: special treatment
        'Transmission:': 'transmission',
        'Engine:': 'cylinders', # Note: special treatment
        'Engine Size:': 'displacement',
        'Driveline:': 'drivetrain',
        'Exterior Color:': 'ext_color', 
        'Interior Color:': 'int_color',
        'Doors:': 'doors', # Note: special treatment
        'Passengers:': 'passengers',
        'Stock Number:': 'stock_id',
        'VIN:': 'vin'
    }
    
    # Fetching the page
    time.sleep(1.1)
    response = requests.get(url)

    # Parsing
    soup = BeautifulSoup(response.text, 'lxml')
    price_node = soup.find('span', class_='PriceValue')
    details_node = soup.find('div', class_='VehicleInfoDetails')
    descr_node = soup.find('div', class_='seller_comments')

    # Basic info
    res = {
        'timestamp': dt.datetime.now().isoformat(),
        'dealer': 'khushiauto',
        'url': url
    }

    # Price
    price = re.sub('[^0-9]', '', price_node.contents[0]) # Cleaning junk, leaving only numbers
    res['price'] = price

    # Reading core parameters
    spans = details_node.find_all('span')

    for s in spans:
        if s.text in translation_table:
            key = translation_table[s.text]
            value = s.next_sibling.text

            # Ad Hoc treatment
            if key in ['cylinders', 'doors']:
                value = value.split()[0]
            elif key == 'mileage':
                value = re.sub('[^0-9]', '', value) # Remove all non numbers using regexp substitution

            res[key] = value
    
    # Free text description        
    #res['description'] = descr.find('p').text

    # Printing result (for debugging purposes)
    print(json.dumps(res, indent=4))
    
    # Done
    return res

In [18]:
# Now scraping vehicle data, one by one
vehicle_data = []

for url in vehicle_urls:
    vehicle_data.append(parse_car_infopage(url))
    
print(len(vehicle_data))

with open('data2/khushiauto_cars.json', 'w', encoding='utf-8') as fp:
    json.dump(vehicle_data, fp, indent=4)

{
    "timestamp": "2020-04-25T08:36:26.813390",
    "dealer": "khushiauto",
    "url": "https://www.khushiauto.ca//cars/used/2016-Nissan-Altima-Sedan-114128",
    "price": "9488",
    "make": "Nissan",
    "model": "Altima",
    "year": "2016",
    "body_style": "Sedan",
    "mileage": "136236",
    "transmission": "Automatic",
    "cylinders": "4",
    "displacement": "2.5",
    "drivetrain": "FWD",
    "ext_color": "Silver",
    "int_color": "Beige",
    "doors": "4",
    "passengers": "-",
    "stock_id": "2624",
    "vin": "1N4AL3AP5GN359677"
}
{
    "timestamp": "2020-04-25T08:36:28.789286",
    "dealer": "khushiauto",
    "url": "https://www.khushiauto.ca//cars/used/2017-Hyundai-Elantra-Sedan-114118",
    "price": "12488",
    "make": "Hyundai",
    "model": "Elantra",
    "year": "2017",
    "body_style": "Sedan",
    "mileage": "42680",
    "transmission": "6 Speed Automatic",
    "cylinders": "4",
    "displacement": "2.0",
    "drivetrain": "FWD",
    "ext_color": "Black",
 

{
    "timestamp": "2020-04-25T08:36:58.036941",
    "dealer": "khushiauto",
    "url": "https://www.khushiauto.ca//cars/used/2015-Chevrolet-Cruze-Sedan-96575",
    "price": "8899",
    "make": "Chevrolet",
    "model": "Cruze",
    "year": "2015",
    "body_style": "Sedan",
    "mileage": "139100",
    "transmission": "Automatic",
    "cylinders": "4",
    "displacement": "1.4",
    "drivetrain": "FWD",
    "ext_color": "Silver",
    "int_color": "Black",
    "doors": "4",
    "passengers": "-",
    "stock_id": "2599",
    "vin": "1G1PC5SB3F7260659"
}
{
    "timestamp": "2020-04-25T08:37:02.002133",
    "dealer": "khushiauto",
    "url": "https://www.khushiauto.ca//cars/used/2016-Nissan-Sentra-Sedan-103066",
    "price": "8999",
    "make": "Nissan",
    "model": "Sentra",
    "year": "2016",
    "body_style": "Sedan",
    "mileage": "128690",
    "transmission": "Automatic",
    "cylinders": "4",
    "displacement": "1.8",
    "drivetrain": "FWD",
    "ext_color": "White",
    "int_c